In [ ]:
!unzip "/content/ml-olympiad-hausa-sentiment-analysis-201.zip"


Archive:  /content/ml-olympiad-hausa-sentiment-analysis-201.zip
replace dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load




from numpy import array

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LOAD DATA**

In [8]:
#Get character encoding of the files
file = ["/content/train.csv", "/content/test.csv"]
for single in file:
    with open(single) as f:
        print(f.encoding)

FileNotFoundError: ignored

In [9]:
train = pd.read_csv('/content/train.csv', encoding='UTF-8')
test = pd.read_csv('/content/test.csv', encoding='UTF-8')

NameError: ignored

**EDA + VISUALIZATIONS**

In [ ]:
#Get summary of the training data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        240 non-null    int64 
 1   NewsText  240 non-null    object
 2   label     240 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 5.8+ KB


In [ ]:
#display first rows of the training dataset
train.head()

,id,NewsText,label
0,1553,Sannan ya ringa yin fina-finai a kan matsalol...,1
1,1059,Karanta cikakken labarin a nan,0
2,1006,Tuni dai Saudiyya ta ce za ta kaddamar da aiki...,1
3,1242,Wannan zai hana raguwar darajarsa kamar yadda ...,-1
4,1221,"Ba shi da wata iyaka, ma'ana iya shafukan da z...",0


In [ ]:
#Get summary of the testing data
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        60 non-null     int64 
 1   NewsText  60 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [ ]:
#display first rows of the testing dataset
test.head()

,id,NewsText
0,1278,"Mene ne sakamakon zaben?""Hukumar zabe ta ayyan..."
1,1721,Gwamnatin Najeriya ta ce an sako yaran ne lami...
2,1180,"""An biya kuɗin fansa kafin a sake su, amma ba ..."
3,1009,Rahotanni sun ce an shafe tsawon sa’a takwas a...
4,1588,Ya karbi lambobin yabo a gida Najeriya da kuma...


# MODELS

**Logistic Regresson**

In [ ]:
#vectorizer:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['NewsText'])
test_matrix = vectorizer.transform(test['NewsText'])

In [ ]:
X_train1 = train_matrix
X_test1 = test_matrix
#y_train = train['sentiment']
#y_test = test['sentiment']

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=10000)
lr_clf.fit(train_features, train_labels)

In [ ]:
y_pred = lr_clf.predict(test_features)
print('Accuracy:', accuracy_score(test_labels, y_pred))
print('Classification Report:\n', classification_report(test_labels, y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
max_k = 40
f1_scores = list()
error_rates = list()

for k in range(1, max_k):
    
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
    knn = knn.fit(train_features, train_labels)
    
    y_pred1 = knn.predict(test_features)
    f1 = f1_score(y_pred, test_labels, average='macro')
    f1_scores.append((k, round(f1_score(test_labels, y_pred1, average='macro'), 4)))
    error = 1-round(accuracy_score(test_labels, y_pred1), 4)
    error_rates.append((k, error))
    
f1_results = pd.DataFrame(f1_scores, columns=['K', 'F1 Score'])
error_results = pd.DataFrame(error_rates, columns=['K', 'Error Rate'])

print(classification_report(test_labels, y_pred1))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

tree_list = [15, 25, 50, 100, 200, 300, 400]

param_grid = {'n_estimators': tree_list,
              'learning_rate': [0.1, 0.01, 0.001, 0.0001],
              'subsample': [1.0, 0.5],
              'max_features': [1, 2, 3, 4]}

GV_GBC = GridSearchCV(GradientBoostingClassifier(random_state=42), 
                      param_grid=param_grid, 
                      scoring='accuracy',
                      n_jobs=-1)

GV_GBC = GV_GBC.fit(train_features, train_labels)

In [ ]:
GV_GBC.best_estimator_

In [ ]:
y_pred2 = GV_GBC.predict(test_features)
print(classification_report(y_pred2, test_labels))

In [ ]:
import seaborn as sns
sns.set_context('talk')
cm = confusion_matrix(test_labels, y_pred2)
ax = sns.heatmap(cm, annot=True, fmt='d')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(oob_score=True, 
                            random_state=42, 
                            warm_start=True,
                            n_jobs=-1)

oob_list = list()

for n_trees in [15, 20, 30, 40, 50, 100, 150, 200, 300, 400]:
    
    RF.set_params(n_estimators=n_trees)

    RF.fit(train_features, train_labels)

    oob_error = 1 - RF.oob_score_
    
    oob_list.append(pd.Series({'n_trees': n_trees, 'oob': oob_error}))

rf_oob_df = pd.concat(oob_list, axis=1).T.set_index('n_trees')

rf_oob_dfsns.set_context('talk')
sns.set_style('white')

ax = rf_oob_df.plot(legend=False, marker='o', figsize=(14, 7), linewidth=5)
ax.set(ylabel='out-of-bag error');

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rff = RandomForestClassifier(n_estimators = 150, n_jobs = 1, random_state =42, warm_start=True)
rff.fit(train_features, train_labels)
y_pred3 = rff.predict(test_features)

print(classification_report(y_pred3, test_labels))


In [ ]:
#The model
from sklearn.ensemble import RandomForestClassifier

lr = RandomForestClassifier(n_estimators=500, random_state=42, max_depth=300)
lr.fit(X_train1,train['label'])

RandomForestClassifier(max_depth=300, n_estimators=500, random_state=42)

In [ ]:
#Training the model
lr.fit(X_train1,train['label'])

RandomForestClassifier(max_depth=300, n_estimators=500, random_state=42)

In [ ]:
#Make predictions
predictions1 = lr.predict(X_test1)
len(predictions1)

60

In [ ]:
print(predictions1[:10])

[ 0  1  0 -1  1  1  0  0  1  0]


Submission File

In [ ]:
sample_submission = pd.read_csv('/content/sample_submission.csv')

In [ ]:
sample_submission.head()

,id,label
0,1278,NaN
1,1721,NaN
2,1180,NaN
3,1009,NaN
4,1588,NaN


In [ ]:
submission = {'id':sample_submission['id'],
              'label':predictions1}

In [ ]:
submission_file = pd.DataFrame(submission)
submission_file.head()

,id,label
0,1278,0
1,1721,1
2,1180,0
3,1009,-1
4,1588,1


In [ ]:
submission_file.shape

(60, 2)

In [ ]:
submission_file.set_index('id', inplace=True)
submission_file.head()

,label
id,
1278,0
1721,1
1180,0
1009,-1
1588,1


In [ ]:
submission_file.to_csv('submission_file8.csv')